In [1]:
from pathlib import Path
from gurobipy import Model, GRB, quicksum


# ----------------- compact formulation -----------------

def solve_compact_formulation(
    V,
    V2,
    E,
    K,
    bar_p,
    hat_p,
    r,
    Nk,
    Gamma,
    M,
    source,
    sink,
    gurobi_output=True,
):
    m = Model("compact_RCPSP")
    if not gurobi_output:
        m.Params.OutputFlag = 0

    S = m.addVars(V, range(Gamma + 1), vtype=GRB.CONTINUOUS, lb=0.0, name="S")
    y = m.addVars(V2, vtype=GRB.BINARY, name="y")
    f = m.addVars(V2, K, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    m.setObjective(S[sink, Gamma], GRB.MINIMIZE)

    m.addConstr(S[source, 0] == 0.0, name="c35_start_time")

    # (36)
    for (i, j) in V2:
        for gamma in range(Gamma + 1):
            m.addConstr(
                S[j, gamma] - S[i, gamma]
                >= bar_p[i] - M * (1 - y[i, j]),
                name=f"c36_timing_nom_{i}_{j}_{gamma}",
            )

    # (37)
    for (i, j) in V2:
        for gamma in range(Gamma):
            m.addConstr(
                S[j, gamma + 1] - S[i, gamma]
                >= bar_p[i] + hat_p[i] - M * (1 - y[i, j]),
                name=f"c37_timing_dev_{i}_{j}_{gamma}",
            )

    # (38)
    for (i, j) in E:
        if (i, j) in y:
            m.addConstr(y[i, j] == 1, name=f"c38_fixed_arc_{i}_{j}")

    # (39)
    for (i, j) in V2:
        for k in K:
            m.addConstr(
                f[i, j, k] <= Nk[k] * y[i, j],
                name=f"c39_capacity_{i}_{j}_{k}",
            )

    # (40)
    for j in V:
        for k in K:
            m.addConstr(
                quicksum(f[i, j, k] for i in V if (i, j) in V2) == r[j, k],
                name=f"c40_flow_in_{j}_{k}",
            )

    # (41)
    for i in V:
        for k in K:
            m.addConstr(
                quicksum(f[i, j, k] for j in V if (i, j) in V2) == r[i, k],
                name=f"c41_flow_out_{i}_{k}",
            )

    # (42)
    for i in V:
        for gamma in range(Gamma + 1):
            m.addConstr(S[i, gamma] >= 0.0, name=f"c42_S_nonneg_{i}_{gamma}")

    # (43)
    for (i, j) in V2:
        for k in K:
            m.addConstr(f[i, j, k] >= 0.0, name=f"c43_f_nonneg_{i}_{j}_{k}")

    m.optimize()
    return m


In [2]:
#mini example

V    = [0,1,2,3,4]          
V2   = [(i,j) for i in V for j in V if i < j]  
E    = [(0,1),(1,2),(2,3),(3,4)] 
K    = [0]

bar_p = {i: 5.0 for i in V}
hat_p = {i: 1.0 for i in V}

r  = {}
for i in V:
    for k in K:
        r[i,k] = 0.0       

Nk = {k: 10.0 for k in K} # N_k ≥ R_k
Gamma = 2
M = 1000.0
source = 0
sink   = 4

model = solve_compact_formulation(
    V, V2, E, K, bar_p, hat_p, r, Nk, Gamma, M, source, sink,
    gurobi_output=True
)
print("Optimal value (S_{n+1,Γ}) =", model.ObjVal)

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 100 rows, 35 columns and 220 nonzeros
Model fingerprint: 0x4acdb71e
Variable types: 25 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 100 rows and 35 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 22 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200000000000e+01, best bound 2.200000000000e+01, gap 0.0000%
Optimal value

In [3]:
from pathlib import Path
from gurobipy import GRB


def _ints(tokens):
    return [int(x) for x in tokens]


def _index_of_line(lines, substr):
    for i, line in enumerate(lines):
        if substr in line:
            return i
    raise ValueError(f"'{substr}' not found in file")


def _rhs_after_colon(line: str) -> str:
    return line.split(":", 1)[1].strip()


def parse_psplib_sm(path):
    """
    Parse a PSPLIB single-mode .sm file into a Python structure:
    graph, durations, resource demands and capacities.
    """
    path = Path(path)
    with path.open() as f:
        lines = [ln.rstrip("\n") for ln in f]

    njobs = int(_rhs_after_colon(lines[_index_of_line(lines, "jobs (incl. supersource")]))
    nres  = int(_rhs_after_colon(lines[_index_of_line(lines, "- renewable")]).split()[0])
    horizon = int(_rhs_after_colon(lines[_index_of_line(lines, "horizon")]))

    prec_offset  = _index_of_line(lines, "PRECEDENCE RELATIONS:") + 2
    attrs_offset = _index_of_line(lines, "REQUESTS/DURATIONS") + 3
    caps_offset  = _index_of_line(lines, "RESOURCEAVAILABILITIES") + 2

    jobs = [f"j{i+1}" for i in range(njobs)]
    resources = [f"r{k+1}" for k in range(nres)]

    succs = {}
    for ix, j in enumerate(jobs):
        tokens = lines[prec_offset + ix].split()
        succ_job_numbers = tokens[3:]
        succs[j] = [f"j{int(s)}" for s in succ_job_numbers]

    durations = {}
    demands = {}

    for ix, j in enumerate(jobs):
        tokens = lines[attrs_offset + ix].split()
        duration = int(tokens[2])
        durations[j] = duration

        res_demands = _ints(tokens[3:])
        for rk, res_name in enumerate(resources):
            demands[(j, res_name)] = res_demands[rk]

    capacity_tokens = lines[caps_offset].split()
    capacities = {resources[k]: int(capacity_tokens[k]) for k in range(nres)}

    return {
        "jobs": jobs,
        "resources": resources,
        "horizon": horizon,
        "succs": succs,
        "durations": durations,
        "demands": demands,
        "capacities": capacities,
    }


def psplib_to_rcpsp_sets(psplib_data):
    """
    Convert PSPLIB data into robust RCPSP input:
    returns V, E, K, bar_p, hat_p, r, Nk,
    where hat_p[i] = bar_p[i] / 2 as in Bold & Goerigk.
    """
    jobs = psplib_data["jobs"]
    resources = psplib_data["resources"]
    succs = psplib_data["succs"]
    durations = psplib_data["durations"]
    demands = psplib_data["demands"]
    capacities = psplib_data["capacities"]

    # activity indices
    job_index = {j: i for i, j in enumerate(jobs)}
    V = list(job_index.values())

    # precedence arcs
    E = []
    for j_from, succ_list in succs.items():
        i = job_index[j_from]
        for j_to in succ_list:
            E.append((i, job_index[j_to]))

    # resources
    K = resources[:]  # e.g. ["r1", "r2", ...]

    # nominal durations and max deviations
    bar_p = {job_index[j]: durations[j] for j in jobs}
    hat_p = {i: bar_p[i] / 2.0 for i in V} #maybe we can do it more clever with a k-mod iterations

    # resource requirements r_{ik}
    r = {}
    for j in jobs:
        i = job_index[j]
        for res in resources:
            r[i, res] = demands[(j, res)]

    # upper bounds N_k ≥ R_k (use capacities directly)
    Nk = {res: capacities[res] for res in resources}

    return V, E, K, bar_p, hat_p, r, Nk


# ----------------- Example: parse rcpsp.sm and solve -----------------
# assumes solve_compact_formulation is defined elsewhere

if __name__ == "__main__":
    data = parse_psplib_sm("rcpsp.sm")

    V, E, K, bar_p, hat_p, r, Nk = psplib_to_rcpsp_sets(data)

    # simple choice for V^2: all i<j pairs
    V2 = [(i, j) for i in V for j in V if i < j]

    source = min(V)
    sink   = max(V)

    # big-M: safe upper bound on makespan
    M = sum(bar_p[i] + hat_p[i] for i in V) + 10.0

    for Gamma in [3, 5, 7]:
        model = solve_compact_formulation(
            V=V,
            V2=V2,
            E=E,
            K=K,
            bar_p=bar_p,
            hat_p=hat_p,
            r=r,
            Nk=Nk,
            Gamma=Gamma,
            M=M,
            source=source,
            sink=sink,
            gurobi_output=True,
        )

        print(f"Gamma = {Gamma}, status = {model.Status}")
        if model.Status == GRB.OPTIMAL:
            print("  Optimal value (S_{n+1,Γ}) =", model.ObjVal)
        else:
            print("  No optimal solution, status code:", model.Status)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
import os

folder = "js30path"   # your folder name

instances = []

for filename in os.listdir(folder):
    if filename.endswith(".sm"):
        fullpath = os.path.join(folder, filename)
        data = parse_psplib_sm(fullpath)
        instances.append((filename, data))

print("Loaded:", len(instances), "instances")


In [4]:
import os
import time
import matplotlib.pyplot as plt
from gurobipy import GRB

# assumes:
#   parse_psplib_sm(path)          -> raw PSPLIB data
#   psplib_to_rcpsp_sets(data)    -> V, E, K, bar_p, hat_p, r, Nk  (robust sets)
#   solve_compact_formulation(...) is already defined

folder = "js30path"   # folder that contains all j30 *.sm files

results = []  # will store one row per (instance, Gamma)

# ---- solve all instances in the folder for Gamma in {3,5,7} ----
for filename in sorted(os.listdir(folder)):
    if not filename.endswith(".sm"):
        continue

    fullpath = os.path.join(folder, filename)
    data = parse_psplib_sm(fullpath)

    V, E, K, bar_p, hat_p, r, Nk = psplib_to_rcpsp_sets(data)

    # V^2: here we simply take all i<j pairs
    V2 = [(i, j) for i in V for j in V if i < j]

    source = min(V)
    sink = max(V)

    # Big-M: safe upper bound on makespan
    M = sum(bar_p[i] + hat_p[i] for i in V) + 10.0

    for Gamma in [3, 5, 7]:
        t0 = time.perf_counter()
        model = solve_compact_formulation(
            V=V,
            V2=V2,
            E=E,
            K=K,
            bar_p=bar_p,
            hat_p=hat_p,
            r=r,
            Nk=Nk,
            Gamma=Gamma,
            M=M,
            source=source,
            sink=sink,
            gurobi_output=False,
        )
        runtime = time.perf_counter() - t0

        status = model.Status
        obj = model.ObjVal if status == GRB.OPTIMAL else None

        results.append(
            {
                "file": filename,
                "Gamma": Gamma,
                "runtime": runtime,
                "status": status,
                "obj": obj,
            }
        )

print("Solved", len(results), "runs")

# ------------------------------------------------------------------
#                   PLOTTING RUNTIME STATISTICS
# ------------------------------------------------------------------

# map each file name to an index for plotting
instance_names = sorted({r["file"] for r in results})
file_to_idx = {name: idx for idx, name in enumerate(instance_names)}

# 1) runtime vs instance index for each Gamma
plt.figure()
for Gamma in [3, 5, 7]:
    xs = []
    ys = []
    for r in results:
        if r["Gamma"] == Gamma and r["status"] == GRB.OPTIMAL:
            xs.append(file_to_idx[r["file"]])
            ys.append(r["runtime"])
    plt.plot(xs, ys, marker="o", linestyle="-", label=f"Gamma={Gamma}")

plt.xlabel("Instance index")
plt.ylabel("Runtime (seconds)")
plt.title("Runtime per instance for different Gamma values")
plt.legend()
plt.tight_layout()
plt.show()

# 2) boxplot of runtimes by Gamma
runtime_by_gamma = []
labels = []
for Gamma in [3, 5, 7]:
    runtimes = [r["runtime"] for r in results
                if r["Gamma"] == Gamma and r["status"] == GRB.OPTIMAL]
    if runtimes:
        runtime_by_gamma.append(runtimes)
        labels.append(f"Gamma={Gamma}")

plt.figure()
plt.boxplot(runtime_by_gamma, labels=labels)
plt.ylabel("Runtime (seconds)")
plt.title("Runtime distribution by Gamma")
plt.tight_layout()
plt.show()


GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information